In [1]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [2]:
import models
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [3]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = models.get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder'

# model is actually in "serialized_models/{model_fname}"
# model_fname = 'rel_tsel_ref_dial_model/default_0_best.th'

# # valid_lang_loss 2.2762        valid_select_loss 0.8293        valid_select_acc 0.6859 valid_ref_loss 0.2897
# model_fname = 'rel2_tsel_ref_dial_model/hierarchical_rel-enc-2_rel-pool-max_mask-pad_1_best.th'

# # valid_lang_loss 2.2609        valid_select_loss 0.9497        valid_select_acc 0.6291         valid_ref_loss 0.3191
# model_fname = 'rel2_tsel_ref_dial_model/hierarchical_rel-enc-2_rel-pool-max_mask-pad_feed-ctx-attend_1_best.th'

# valid_lang_loss 
# # valid_lang_loss 2.2544        valid_select_loss 0.8247        valid_select_acc 0.7062         valid_ref_loss 0.2852
# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_feed-context-attend-separate_1_best.th'

# # valid_lang_loss 2.2448        valid_select_loss 0.9233        valid_select_acc 0.6455         valid_ref_loss 0.3180
# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_selection-attention_feed-context-attend-separate_no-word-attention_1_best.th'

# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_selection-attention_feed-context-attend-separate_1_best.th'

# valid_lang_loss 2.2435        valid_select_loss 1.0267        valid_select_acc 0.6358         valid_ref_loss 0.3528
# model_fname = 'rel2_tsel_ref_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'

model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

In [4]:
seed = 0
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

dataset data/onecommon/train_reference_0.txt, total 544674, unks 4134, ratio 0.76%
dataset data/onecommon/valid_reference_0.txt, total 70004, unks 720, ratio 1.03%
dataset data/onecommon/test_reference_0.txt, total 67008, unks 576, ratio 0.86%


In [5]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [6]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [7]:
len(scenarios1)

9000

In [8]:
len(scenarios2)

4500

In [9]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [10]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [11]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [12]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [13]:
corpus

In [14]:
model = utils.load_model(model_fname, map_location='cpu')

In [15]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

In [16]:
validset_stats

{'n': 74862, 'nonpadn': 70004}

In [17]:
import torch

In [18]:
from agent import RnnAgent

/home/dfried/anaconda2/envs/onecommon/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [19]:
from selfplay import make_parser

In [20]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [21]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [22]:
def is_selection(out):
    return '<selection>' in out

In [23]:
# display

In [24]:
from dialog import DialogLogger

In [25]:
from IPython.display import SVG, display, HTML

In [26]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [27]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn)])

In [28]:
pdb on

Automatic pdb calling has been turned ON


In [29]:
# training emulation

In [30]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

In [31]:
trainset[0].scenario_ids[0] in train_scenarios

False

In [32]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [37]:
for ix in range(len(trainset)):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
        print("found it")
        break

found it


In [38]:
inpts[0].size()

torch.Size([16, 1])

In [39]:
corpus.word_dict

In [41]:
## training emulation
# scenario = train_scenarios[scenario_id]
# agent = alice
# agent.feed_context(ctx.flatten())
# agent.real_ids = real_ids
# agent.agent_id = 0
    
# samples = 10
    
# writer = alice
# conv = []
# speaker = []
# words_left = 5000

# sentence_ix = 0

# assert len(inpts) == len(tgts)
    
# while sentence_ix < len(inpts):
#     words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
# #     for _ in range(samples):
# #         pred_outs, _, _, _, extra = writer.model.write(writer.ctx_h, writer.lang_h, words_left, writer.args.temperature)
# #         print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
#     print(' '.join(words_og))
#     out = writer.write(max_words=words_left, force_words=[words_og], start_token=words_og[0])
# #     out = writer.write(max_words=words_left)
    
#     display_attn(scenario, writer.extras[-1]['dot_attention1'], writer.agent_id, name='feed attention')
    
#     if 'dot_attention2_mean' in writer.extras[-1]:
#         display_attn(scenario, writer.extras[-1]['dot_attention2_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['dot_attention2s']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
#     print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
#     print()
#     words_left -= len(out)
#     conv.append(out)
#     speaker.append(writer.agent_id)
# #     reader.read(out)
#     if is_selection(out) or words_left <= 1:
#         break
# #     writer, reader = reader, writer
#     sentence_ix += 1

In [42]:
import json
from utils import ContextGenerator

In [43]:
ctx_gen = ContextGenerator('data/onecommon/shared_5.txt')

In [44]:
with open('data/onecommon/shared_5.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [45]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 1:
        scenario = selfplay_scenarios[scenario_id]
        break

In [46]:
print(scenario_id)

S_KtpcMNhRP3KfKgxQ


In [50]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
samples = 0
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000
    
while True:
    for _ in range(samples):
        outs, _, _, _, extra = writer.model.write(writer.ctx_h, writer.lang_h, words_left, writer.args.temperature)
        print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
    out = writer.write(max_words=words_left)
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
    display_attn(scenario, writer.extras[-1]['dot_attention1'], writer.agent_id, name='feed attention')
    
    if 'dot_attention2_mean' in writer.extras[-1]:
        display_attn(scenario, writer.extras[-1]['dot_attention2_mean'], writer.agent_id, name='word attention mean')
        
    for ix, ix_attn in enumerate(writer.extras[-1]['dot_attention2s']):
        display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

0	i have a large black dot with a smaller light grey dot to the right of it <eos>

feed attention: [4.0425375e-01 2.2526349e-06 4.8737253e-10 4.8045615e-16 5.5356938e-01
 4.0691510e-02 1.4831466e-03]


word attention mean: [0.21453267 0.1086643  0.04040901 0.00164877 0.26169434 0.21964608
 0.1534048 ]


t 0: i: [2.1879005e-01 1.3062571e-01 6.7302119e-03 7.5696671e-06 2.2003511e-01
 2.1608242e-01 2.0772897e-01]


t 1: have: [2.6443505e-01 2.2380693e-02 3.9239624e-04 3.1872315e-07 3.1523326e-01
 2.6885673e-01 1.2870154e-01]


t 2: a: [2.1130657e-01 1.4445600e-01 3.1705108e-02 1.1004100e-04 2.1389352e-01
 2.0922853e-01 1.8930018e-01]


...

t 7: the: [2.1123382e-01 1.4195952e-01 2.5406213e-02 8.0475373e-05 2.0908077e-01
 2.1052511e-01 2.0171405e-01]


t 8: black: [2.4333137e-01 4.1556042e-03 1.9186350e-04 5.8719991e-07 4.3349433e-01
 2.8115144e-01 3.7674773e-02]


t 9: one: [2.6116040e-01 3.5355806e-02 3.5511791e-03 3.0202800e-05 3.1695914e-01
 2.6227555e-01 1.2066770e-01]


t 10: <selection>: [0.18623145 0.16384558 0.08672152 0.00083736 0.18378927 0.18618007
 0.19239475]


0	i have that . let 's pick the black one <selection>

